In [1]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn

from util.ZigZag import ZigZag
from util.BuildData import BuildTrainSetForMLUsingPrices,  BuildTrainSetForMLUsingReturns
from util.CalcReturns import CalcReturns

from sklearn import svm, preprocessing

In [2]:
%matplotlib qt

In [3]:
def recurrent_neural_network_model(x):

    layer = {'w' : tf.Variable(tf.random_normal([num_hidden, n_classes])),
             'b' : tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.unstack(x, timesteps, 1)    
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)    
    outs, state = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)        
    y = tf.add(tf.matmul(outs[-1], layer['w']), layer['b'], name='nn')
    
    return y

In [4]:
def train_and_save_nn(x,y):
    nn = recurrent_neural_network_model(x)
    
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = nn, labels = y))
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(cost)   
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.0006).minimize(cost)   
    
    correct = tf.equal(tf.argmax(nn,1), tf.argmax(y_train, 1), name='correct')
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name = 'accuracy')
    
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())

        epoch_loss = 0
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int((len(D) - t_count)/batch_size)):
                
                batch_x = D[i*batch_size : (i+1)*batch_size]
                batch_y = L[i*batch_size : (i+1)*batch_size]
               
                batch_x = batch_x.reshape((batch_size, timesteps, num_input))
                
                _, c = session.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
                epoch_loss += c
                
            if epoch % 100 == 0:
                print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
        
        
        
                
        saver.save(session, 'data/fxnn.ckpt')
        
        print('Last epoch loss: ', epoch_loss)
        return nn, correct, accuracy

In [5]:
def test_nn(x_test, y_test):
    with tf.Session() as session:
        saver = tf.train.import_meta_graph('data/fxnn.ckpt.meta')
        saver.restore(session, tf.train.latest_checkpoint('data/'))
        
        graph    = tf.get_default_graph()
        correct  = graph.get_tensor_by_name('correct:0')
        accuracy = graph.get_tensor_by_name('accuracy:0')
        
        x = graph.get_tensor_by_name('x:0')
        y = graph.get_tensor_by_name('y:0')

        x_test = x_test.reshape((t_count, timesteps, num_input))
        
        r = tf.cast(correct, 'float').eval({x:x_test, y:y_test})       

        #for i in range(len(r)):
        #    print '%s %s' % (r[i], y_test[i])        

        test_buy  = 0.0
        test_sell = 0.0

        correct_buy  = 0.0
        correct_sell = 0.0

        incorrect_buy  = 0.0
        incorrect_sell = 0.0

        for i in range(len(r)):
            if y_test[i][1] == 1:
                test_buy += 1.0;
            elif y_test[i][0] == 1:
                test_sell += 1.0;

            if r[i] == 1:
                if y_test[i][1] == 1:
                    correct_buy += 1.0;
                elif y_test[i][0] == 1:
                    correct_sell += 1.0;
            else:
                if y_test[i][1] == 1:
                    incorrect_buy += 1.0;
                elif y_test[i][0] == 1:
                    incorrect_sell += 1.0;

        print('Accuracy:',accuracy.eval({x:x_test, y:y_test}))

        print( "Test buy:  ", test_buy  )
        print( "Test sell: ", test_sell )

        print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
        print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )

In [6]:
n_classes  = 2

step = 61

timesteps = 10
num_input = 6

#step - 1 == num_input * timesteps

hm_epochs  = 1000
num_hidden = 16
batch_size = 100
t_count = 1024


##########################################################################

source = pd.read_csv('EURUSD1.csv', header=0) # source CSV
prices = np.array(source.Close) # close prices

#Z = ZigZag(P, 0.0064) # build ZigZag

T, S = BuildTrainSetForMLUsingReturns(prices, 0.0064, 60)


#T, _ = BuildData0(Z, P) # prepare training set

#plt.plot(T["input"])


#dims = T["input"].shape

#print dims

#D = S["data"]
#L = []

#for i in S["label"]:
#    if i == 1:
#        L.append([0, 1])
#    elif i == -1:
#        L.append([1, 0])

#L = np.array(L)
        
##print D.shape
##print L.shape

#x_train = tf.placeholder('float', [None, timesteps, num_input], name='x')
#y_train = tf.placeholder('float', [None, n_classes], name='y')

#train_and_save_nn(x_train, y_train)

815 1049 234 1 0.00649 1.17489 1.18138 0.00649
1049 1758 709 0 0 1.18138 1.18115 -0.00023
1758 2240 482 -1 -0.00642 1.18115 1.17473 -0.00642
2240 3486 1246 0 0 1.17473 1.17439 -0.00034
3486 3889 403 1 0.00655 1.17439 1.18094 0.00655
3889 6753 2864 0 0 1.18094 1.17933 -0.00161
6753 6756 3 1 0.00663 1.17933 1.18596 0.00663
6756 7052 296 0 0 1.18596 1.18771 0.00175
7052 7174 122 1 0.0066 1.18771 1.19431 0.0066
7174 7663 489 0 0 1.19431 1.19185 -0.00246
7663 8447 784 1 0.00643 1.19185 1.19828 0.00643
8447 9207 760 0 0 1.19828 1.19956 0.00128
9207 9277 70 1 0.00651 1.19956 1.20607 0.00651
9277 9819 542 0 0 1.20607 1.20125 -0.00482
9819 9995 176 -1 -0.00652 1.20125 1.19473 -0.00652
9995 10708 713 0 0 1.19473 1.19594 0.00121
10708 10982 274 -1 -0.00641 1.19594 1.18953 -0.00641
10982 11046 64 0 0 1.18953 1.19356 0.00403
11046 11990 944 -1 -0.00642 1.19356 1.18714 -0.00642
11990 12251 261 0 0 1.18714 1.18915 0.00201
12251 12403 152 -1 -0.00645 1.18915 1.1827 -0.00645
12403 12464 61 0 0 1.1827 1

/usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(14306, 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(234, 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(709, 0) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(482, -1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(1246, 0) will return an arra

In [ ]:
x_test = D[len(D)-t_count:]
y_test = L[len(L)-t_count:]

test_nn(x_test, y_test)

In [3]:
from scipy.linalg import hankel
L = [0, 1, 2, 3, 4, 5, 6]
hankel(L[0:4], L[3:])

array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [2, 3, 4, 5],
       [3, 4, 5, 6]])